# Solution Animation: 2D

## Description
Notebook of optimization algorithm solution animations superimposed on 2d filled contour plots.

In [1]:
import json
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.animation import FuncAnimation, FFMpegFileWriter

from IPython.display import HTML

%matplotlib inline

## Test Functions

In [2]:
def rosenbrock(x):
    """
    rosenbrock evaluates Rosenbrock function at vector x

    Parameters
    ----------
    x : array
        x is a D-dimensional vector, [x1, x2, ..., xD]

    Returns
    -------
    float
        scalar result
    """
    D = len(x)
    i, iplus1 = np.arange(0,D-1), np.arange(1,D)
    return np.sum(100*(x[iplus1] - x[i]**2)**2 + (1-x[i])**2)


def goldstein_price(x):
    """
    goldstein_price evaluates Goldstein-Price function at vector x

    Parameters
    ----------
    x : array
        x is a 2-dimensional vector, [x1, x2]

    Returns
    -------
    float
        scalar result
    """
    a = (x[0] + x[1] + 1)**2
    b = 19 - 14*x[0] + 3*x[0]**2 - 14*x[1] + 6*x[0]*x[1] + 3*x[1]**2
    c = (2*x[0] - 3*x[1])**2
    d = 18 - 32*x[0] + 12*x[0]**2 + 48*x[1] - 36*x[0]*x[1] + 27*x[1]**2
    return (1. + a*b) * (30. + c*d)


def bartels_conn(x):
    """
    bartels_conn evaluates Bartels-Conn function at vector x

    Parameters
    ----------
    x : array
        x is a 2-dimensional vector, [x1, x2]

    Returns
    -------
    float
        scalar result
    """
    a = np.abs(x[0]**2 + x[1]**2 + x[0]*x[1])
    b = np.abs(np.sin(x[0]))
    c = np.abs(np.cos(x[1]))
    return a + b +c


def egg_crate(x):
    """
    egg_crate evaluates Egg Crate function at vector x

    Parameters
    ----------
    x : array
        x is a 2-dimensional vector, [x1, x2]

    Returns
    -------
    float
        scalar result
    """
    return x[0]**2 + x[1]**2 + 25.*(np.sin(x[0])**2 + np.sin(x[1])**2)

## Surface Generation

In [3]:
def surface(fx, start=-30, stop=30, num=60):
    """
    surface evaluates fx at regularly spaced grid of points

    Parameters
    ----------
    fx : func
        fx is a vector valued function that returns a scalar result
    start : float
        lower bound of the coordinate grid
    stop : float
        upper bound of the coordinate grid
    num : int
        number of points along one dimension of the grid

    Returns
    -------
    array
        2D array formed by evaluating fx at each grid point
    """
    x = np.linspace(start=start, stop=stop, num=num)
    x1, x2 = np.meshgrid(x, x, indexing='ij')
    X = np.vstack((x1.ravel(), x2.ravel()))
    z = np.apply_along_axis(fx, 0, X).reshape(num,num)
    return x1, x2, z

## Solution Results

In [4]:
def load_steps(**params):
    """Return solution steps based on simulation properties."""
    savefn = os.path.join(params['base_dirn'],
                          params['savefn_fmt'].format(**params))
    return np.load(savefn)


def load_meta(**params):
    """Return metafile based on simulation properties."""
    metafn = os.path.join(params['base_dirn'],
                          params['metafn_fmt'].format(**params))
    return json.load(open(metafn, 'r'))


def anim2d_solutions(**params):
    """
    anim2d_solutions creates 2d animation from simulation results
    """
    algstr = params['alg'].replace('_',' ').title()
    funcstr = params['func'].replace('_',' ').title()
    ngridpts = params.get('ngridpts', 500)
    bounds = params['bounds']
    trial = params['trial']  # Single trial only.
    xkmind = params.get('xkmind', slice(2))
    color = params.get('color', 'darkorange')
    ticker_locator = params.get('ticker_locator', 'LinearLocator')
    colorbar_label = params.get('colorbar_label', 'z')
    fps = params.get('fps', 30)
    bitrate = params.get('bitrate', 5000)
    show_legend = params.get('show_legend', True)
    
    # Imbue title with simulation meta information.
    meta = load_meta(**params)
    expmin, expxkmin = meta['exp_fxkmin'], meta['exp_xkmin']
    expminstr = 'abs $\\min(f)$={0:.0f}'.format(expmin)
    algstr = algstr if len(algstr) > 4 else algstr.upper()
    algmeta = [('nx0','n'),('T0','$T_0$'),
               ('alpha','$\\alpha$'),('tol','tol')]
    algmetastr = ' '.join(['{0}={1}'.format(n2, meta[n1])
                           for n1, n2 in algmeta if n1 in meta])
    nitstr = 'nit={0:d}'.format(meta['nsteps'][trial-1])
    minfx = meta['f(xk)'][trial-1]
    minfmt = '.2e' if minfx < 1e-1 else '.1f'
    minstr = '$\\min(f)$={0:{1}}'.format(minfx, minfmt)
    metastrs = [algstr, minstr, nitstr, algmetastr]
    titlestr = ' '.join([s for s in metastrs if len(s) > 0])
    suptitlestr = 'Solution Trajectories: {0} Function'.format(funcstr)

    # Generate surface for filled contour plot.
    fx = globals()[params['func']]
    start, stop = np.min(bounds[::2]), np.max(bounds[1::2])
    x1, x2, z = surface(fx, start, stop, ngridpts)

    fig = plt.figure(figsize=(8,6))

    # Plot 2d filled contour.
    locator = getattr(ticker, ticker_locator)
    cs = plt.contourf(x1, x2, z, locator=locator(), cmap='viridis_r',
                      alpha=0.7)

    # Plot expected minimum.
    plt.scatter(expxkmin[0], expxkmin[1], marker='D', c='red', s=30,
                label=expminstr)

    # Plot initial point.
    x0 = np.array(meta['x0'][trial-1]).reshape(-1,2)
    plt.scatter(x0[:,0], x0[:,1], marker='X', c='dodgerblue', s=30,
                label='$x_0$')

    # Plot bounds.
    plt.suptitle(suptitlestr)
    plt.title(titlestr)
    plt.xlabel('x1')
    plt.xlim(bounds[:2])
    plt.ylabel('x2')
    plt.ylim(bounds[2:])
    plt.colorbar(cs, label=colorbar_label)

    # Load solution trajectory.
    steps = load_steps(**params)
    xks = steps[:,xkmind]
    xks = np.clip(xks, a_min=bounds[::2], a_max=bounds[1::2])
    nxks = 0 if np.isnan(xks).any() else len(xks) 

    ln, = plt.plot([], [],
                   ls=(0, (1,1)), lw=2, c=color,
                   label='$x_k$, trial={:d}'.format(trial))

    def update(ind):
        ln.set_data(xks[:ind+1,0], xks[:ind+1,1])
        return ln,

    if show_legend:
        plt.legend()

    anim = FuncAnimation(fig, update, frames=nxks, blit=True)
    if params.get('anim2dfn_fmt') is not None:             
        imgn = params['anim2dfn_fmt'].format(**params)
        animfn = os.path.join(params['base_dirn'], imgn)
        writer = FFMpegFileWriter(fps=fps, bitrate=bitrate)
        anim.save(animfn, writer=writer)
    plt.close(fig)
    return anim

## Solutions: Rosenbrock Function

In [5]:
params = {
    'alg': 'gradient_descent',
    'func': 'rosenbrock',
    'trial': 1,
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [6]:
params = {
    'alg': 'bfgs',
    'func': 'rosenbrock',
    'trial': 1,
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [7]:
params = {
    'alg': 'simulated_annealing',
    'func': 'rosenbrock',
    'trial': 7,
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'xkmind': slice(3,5),
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [8]:
params = {
    'alg': 'particle_swarm',
    'func': 'rosenbrock',
    'trials': [1],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'xkmind': slice(4,6),
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

## Solutions: Goldstein-Price Function

In [9]:
params = {
    'alg': 'gradient_descent',
    'func': 'goldstein_price',
    'trials': [5],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [10]:
params = {
    'alg': 'bfgs',
    'func': 'goldstein_price',
    'trials': [5],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [11]:
params = {
    'alg': 'simulated_annealing',
    'func': 'goldstein_price',
    'trials': [4],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'xkmind': slice(3,5),
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [12]:
params = {
    'alg': 'particle_swarm',
    'func': 'goldstein_price',
    'trials': [1],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-2.,2.,-2.,2.],
    'xkmind': slice(4,6),
    'ticker_locator': 'LogLocator',
    'colorbar_label': 'log(z)',
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

## Solutions: Bartels-Conn

In [13]:
params = {
    'alg': 'simulated_annealing',
    'func': 'bartels_conn',
    'trials': [3],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-5.,5.,-5.,5.],
    'xkmind': slice(3,5),
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [14]:
params = {
    'alg': 'particle_swarm',
    'func': 'bartels_conn',
    'trials': [1],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-5.,5.,-5.,5.],
    'xkmind': slice(4,6),
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

## Solutions: Egg Crate

In [15]:
params = {
    'alg': 'simulated_annealing',
    'func': 'egg_crate',
    'trial': 4,
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-5.,5.,-5.,5.],
    'xkmind': slice(3,5),
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())

In [16]:
params = {
    'alg': 'particle_swarm',
    'func': 'egg_crate',
    'trials': [1],
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'bounds': [-5.,5.,-5.,5.],
    'xkmind': slice(4,6),
}

#anim = anim2d_solutions(**params)
#HTML(anim.to_jshtml())